## Modeling

I decided to be bold and use a pretrained BERT model. Then I used a text feature based on combining the title and body, and a set of engineered features. From BERT I used the tokanization and embeddings. Then I built my own regression model for the head.

I choose to use a few layers deep NN. Depth should help with the non-linearities and multi-modalities. The presence of the (scaled) numerical feature is mean to help bring down the bias. I used the Huber loss (SmoothL1Loss) for robust regression.

Given I am working on an older M1 Macbook air, computations were slow. So I did not get to do much hyper-parameter tuning. 

Another idea that could have helped with faster computations (but I thought about it late) is to actually download the BERT embeddings for our dataset then treat them like features, combine them with the numerical features and use them together in a regression model to predict the score.

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR

from sklearn.model_selection import train_test_split
from sklearn.metrics import (mean_absolute_error, 
                             mean_squared_error, 
                             mean_absolute_percentage_error,
                             r2_score)

from utils4 import *

In [2]:
# load the data
df = load_data('data_features.csv')
# Split into train/val/test sets (70/15/15)
df_train, df_val, df_test = prepare_data(df, test_size=.15)

In [3]:
# tokenize the data
train_encodings, train_labels, train_numerical_features = tokenize_data(
    df_train, df_train[NUMERICAL_FEATURES].values)
val_encodings, val_labels, val_numerical_features = tokenize_data(
    df_val, df_val[NUMERICAL_FEATURES].values)
test_encodings, test_labels, test_numerical_features = tokenize_data(
    df_test, df_test[NUMERICAL_FEATURES].values)

In [4]:
# create the custom datasets with numerical features
train_dataset = TextDataset(train_encodings, train_labels, train_numerical_features)
val_dataset = TextDataset(val_encodings, val_labels, val_numerical_features)
test_dataset = TextDataset(test_encodings, test_labels, test_numerical_features)


In [5]:
# create the data loaders
batch_size = 32
train_loader, val_loader, test_loader = create_data_loaders(train_dataset, 
                                                            val_dataset, 
                                                            test_dataset, 
                                                            batch_size)

In [6]:
# create the RedditRegression model instance and move it to the device
model = RedditRegression(num_numerical_features=len(NUMERICAL_FEATURES)).to(device)

# freeze the pre-trained weights
for param in model.bert.parameters():
    param.requires_grad = False

# define the loss function (Huber)
loss_fn = nn.SmoothL1Loss(reduction='mean')

# define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=.01)

# define the scheduler
scheduler = StepLR(optimizer, step_size=3, gamma=0.1)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
num_epochs = 10
original_scale = False
for epoch in tqdm(range(num_epochs), desc='Epochs'):
    if epoch > 4:
        original_scale = True
    # train
    train_loss, train_mae, train_mape = train(model,
                                              train_loader, 
                                              loss_fn, 
                                              optimizer, 
                                              original_scale)
    print(
        f'Epoch {epoch+1}, Train Loss: {train_loss}, Train MAE: {train_mae}, Train MAPE: {train_mape}')
    scheduler.step()

    # evaluation on validation set
    val_loss, val_mae, val_mape, val_labels, val_yhats = evaluate(model,
                                                                  val_loader,
                                                                  loss_fn,
                                                                  original_scale)
    print(
        f'Epoch {epoch+1}, Val Loss: {val_loss}, Val MAE: {val_mae}, Val MAPE: {val_mape}')

    # evaluation on test set
    test_loss, test_mae, test_mape, test_labels, test_yhats = evaluate(model, 
                                                                       test_loader, 
                                                                       loss_fn, 
                                                                       original_scale)
    print(
        f'Epoch {epoch+1}, Test Loss: {test_loss}, Test MAE: {test_mae}, Test MAPE: {test_mape}')


Training:  16%|▎ | 15/95 [01:01<05:23,  4.04s/it, Loss=1.18, MAE=1.57, MAPE=1.4]
Training:  16%|▏| 15/95 [01:05<05:23,  4.04s/it, Loss=0.973, MAE=1.35, MAPE=1.07
Training:  17%|▏| 16/95 [01:05<05:19,  4.04s/it, Loss=0.973, MAE=1.35, MAPE=1.07
Training:  20%|▏| 19/95 [01:17<05:07,  4.04s/it, Loss=1.32, MAE=1.73, MAPE=1.98]
Training:  20%|▏| 19/95 [01:21<05:07,  4.04s/it, Loss=0.917, MAE=1.32, MAPE=1.06
Training:  21%|▏| 20/95 [01:21<05:03,  4.04s/it, Loss=0.917, MAE=1.32, MAPE=1.06
Training:  22%|▏| 21/95 [01:25<04:59,  4.04s/it, Loss=2.54, MAE=2.95, MAPE=1.15]
Training:  22%|▏| 21/95 [01:29<04:59,  4.04s/it, Loss=1.02, MAE=1.46, MAPE=0.751
Training:  23%|▏| 22/95 [01:29<04:55,  4.05s/it, Loss=1.02, MAE=1.46, MAPE=0.751
Training:  32%|▎| 30/95 [02:01<04:24,  4.07s/it, Loss=1.37, MAE=1.8, MAPE=0.897]
Training:  32%|▎| 30/95 [02:05<04:24,  4.07s/it, Loss=1.66, MAE=2.12, MAPE=0.966
Training:  33%|▎| 31/95 [02:05<04:20,  4.07s/it, Loss=1.66, MAE=2.12, MAPE=0.966
Training:  34%|▎| 32/95 [02:

Epoch 1, Train Loss: 6.709235147426003, Train MAE: 7.147599697113037, Train MAPE: 3.98036527633667



Evaluation:   0%|       | 0/17 [00:03<?, ?it/s, Loss=0.949, MAE=1.43, MAPE=2.08]
Evaluation:   6%| | 1/17 [00:03<00:48,  3.05s/it, Loss=0.949, MAE=1.43, MAPE=2.0
Evaluation:   6%| | 1/17 [00:06<00:48,  3.05s/it, Loss=1.21, MAE=1.69, MAPE=1.96
Evaluation:  12%| | 2/17 [00:06<00:46,  3.07s/it, Loss=1.21, MAE=1.69, MAPE=1.96
Evaluation:  12%| | 2/17 [00:09<00:46,  3.07s/it, Loss=1.18, MAE=1.67, MAPE=2.05
Evaluation:  18%|▏| 3/17 [00:09<00:43,  3.08s/it, Loss=1.18, MAE=1.67, MAPE=2.05
Evaluation:  18%|▏| 3/17 [00:12<00:43,  3.08s/it, Loss=0.858, MAE=1.3, MAPE=1.07
Evaluation:  24%|▏| 4/17 [00:12<00:40,  3.09s/it, Loss=0.858, MAE=1.3, MAPE=1.07
Evaluation:  24%|▏| 4/17 [00:15<00:40,  3.09s/it, Loss=1.11, MAE=1.59, MAPE=2.14
Evaluation:  29%|▎| 5/17 [00:15<00:37,  3.09s/it, Loss=1.11, MAE=1.59, MAPE=2.14
Evaluation:  29%|▎| 5/17 [00:18<00:37,  3.09s/it, Loss=0.976, MAE=1.41, MAPE=1.7
Evaluation:  35%|▎| 6/17 [00:18<00:34,  3.09s/it, Loss=0.976, MAE=1.41, MAPE=1.7
Evaluation:  47%|▍| 8/17 [0

Epoch 1, Val Loss: 1.047254502773285, Val MAE: 1.510587215423584, Val MAPE: 1.8130682706832886



Evaluation:   0%|        | 0/20 [00:03<?, ?it/s, Loss=0.94, MAE=1.36, MAPE=2.26]
Evaluation:   5%| | 1/20 [00:03<00:58,  3.10s/it, Loss=0.94, MAE=1.36, MAPE=2.26
Evaluation:   5%| | 1/20 [00:06<00:58,  3.10s/it, Loss=0.944, MAE=1.39, MAPE=2.1
Evaluation:  10%| | 2/20 [00:06<00:55,  3.10s/it, Loss=0.944, MAE=1.39, MAPE=2.1
Evaluation:  10%| | 2/20 [00:09<00:55,  3.10s/it, Loss=0.988, MAE=1.45, MAPE=1.9
Evaluation:  15%|▏| 3/20 [00:09<00:52,  3.10s/it, Loss=0.988, MAE=1.45, MAPE=1.9
Evaluation:  15%|▏| 3/20 [00:12<00:52,  3.10s/it, Loss=0.971, MAE=1.43, MAPE=1.8
Evaluation:  20%|▏| 4/20 [00:12<00:49,  3.12s/it, Loss=0.971, MAE=1.43, MAPE=1.8
Evaluation:  20%|▏| 4/20 [00:15<00:49,  3.12s/it, Loss=1.09, MAE=1.56, MAPE=2.07
Evaluation:  25%|▎| 5/20 [00:15<00:46,  3.11s/it, Loss=1.09, MAE=1.56, MAPE=2.07
Evaluation:  30%|▎| 6/20 [00:18<00:43,  3.11s/it, Loss=1.06, MAE=1.5, MAPE=2.21]
Evaluation:  30%|▎| 6/20 [00:21<00:43,  3.11s/it, Loss=1.11, MAE=1.56, MAPE=1.42
Evaluation:  35%|▎| 7/20 [0

...


Evaluation:   0%| | 0/17 [00:03<?, ?it/s, Loss=0.736, MAE=3.61e+3, MAPE=1.79e+8]
Evaluation:   6%| | 1/17 [00:03<00:51,  3.24s/it, Loss=0.736, MAE=3.61e+3, MAPE=
Evaluation:   6%| | 1/17 [00:06<00:51,  3.24s/it, Loss=0.74, MAE=3.6e+3, MAPE=2.
Evaluation:  12%| | 2/17 [00:06<00:48,  3.24s/it, Loss=0.74, MAE=3.6e+3, MAPE=2.
Evaluation:  12%| | 2/17 [00:09<00:48,  3.24s/it, Loss=0.873, MAE=3.31e+3, MAPE=
Evaluation:  18%|▏| 3/17 [00:09<00:45,  3.23s/it, Loss=0.873, MAE=3.31e+3, MAPE=
Evaluation:  18%|▏| 3/17 [00:12<00:45,  3.23s/it, Loss=0.511, MAE=2.86e+3, MAPE=
Evaluation:  24%|▏| 4/17 [00:12<00:41,  3.22s/it, Loss=0.511, MAE=2.86e+3, MAPE=
Evaluation:  24%|▏| 4/17 [00:16<00:41,  3.22s/it, Loss=0.876, MAE=5.2e+3, MAPE=3
Evaluation:  29%|▎| 5/17 [00:16<00:38,  3.22s/it, Loss=0.876, MAE=5.2e+3, MAPE=3
Evaluation:  29%|▎| 5/17 [00:19<00:38,  3.22s/it, Loss=0.794, MAE=1.75e+3, MAPE=
Evaluation:  35%|▎| 6/17 [00:19<00:35,  3.22s/it, Loss=0.794, MAE=1.75e+3, MAPE=
Evaluation:  35%|▎| 6/17 [0

Epoch 9, Val Loss: 0.7442489546888015, Val MAE: 3191.43505859375, Val MAPE: 1101428480.0



Evaluation:   0%|   | 0/20 [00:03<?, ?it/s, Loss=0.8, MAE=2.77e+3, MAPE=3.48e+9]
Evaluation:   5%| | 1/20 [00:03<01:01,  3.22s/it, Loss=0.8, MAE=2.77e+3, MAPE=3.
Evaluation:   5%| | 1/20 [00:06<01:01,  3.22s/it, Loss=0.69, MAE=4.34e+3, MAPE=3
Evaluation:  10%| | 2/20 [00:06<00:58,  3.23s/it, Loss=0.69, MAE=4.34e+3, MAPE=3
Evaluation:  10%| | 2/20 [00:09<00:58,  3.23s/it, Loss=0.909, MAE=2.84e+3, MAPE=
Evaluation:  15%|▏| 3/20 [00:09<00:54,  3.23s/it, Loss=0.909, MAE=2.84e+3, MAPE=
Evaluation:  15%|▏| 3/20 [00:12<00:54,  3.23s/it, Loss=0.769, MAE=2.99e+3, MAPE=
Evaluation:  20%|▏| 4/20 [00:12<00:51,  3.23s/it, Loss=0.769, MAE=2.99e+3, MAPE=
Evaluation:  20%|▏| 4/20 [00:16<00:51,  3.23s/it, Loss=0.72, MAE=2.33e+3, MAPE=7
Evaluation:  25%|▎| 5/20 [00:16<00:48,  3.22s/it, Loss=0.72, MAE=2.33e+3, MAPE=7
Evaluation:  25%|▎| 5/20 [00:19<00:48,  3.22s/it, Loss=0.922, MAE=4.22e+3, MAPE=
Evaluation:  30%|▎| 6/20 [00:19<00:45,  3.23s/it, Loss=0.922, MAE=4.22e+3, MAPE=
Evaluation:  30%|▎| 6/20 [0

Epoch 9, Test Loss: 0.7492606431245804, Test MAE: 2999.86572265625, Test MAPE: 3245786624.0



Training:   0%|    | 0/95 [00:04<?, ?it/s, Loss=0.61, MAE=1.21e+3, MAPE=6.74e+9]
Training:   1%| | 1/95 [00:04<06:42,  4.28s/it, Loss=0.61, MAE=1.21e+3, MAPE=6.7
Training:   1%| | 1/95 [00:08<06:42,  4.28s/it, Loss=0.918, MAE=2.79e+3, MAPE=1.
Training:   2%| | 2/95 [00:08<06:35,  4.26s/it, Loss=0.918, MAE=2.79e+3, MAPE=1.
Training:   2%| | 2/95 [00:12<06:35,  4.26s/it, Loss=0.635, MAE=3151.07, MAPE=4.
Training:   3%| | 3/95 [00:12<06:32,  4.26s/it, Loss=0.635, MAE=3151.07, MAPE=4.
Training:   3%| | 3/95 [00:17<06:32,  4.26s/it, Loss=1.02, MAE=3.17e+3, MAPE=2.9
Training:   4%| | 4/95 [00:17<06:27,  4.26s/it, Loss=1.02, MAE=3.17e+3, MAPE=2.9
Training:   4%| | 4/95 [00:21<06:27,  4.26s/it, Loss=1.02, MAE=4.56e+3, MAPE=7.2
Training:   5%| | 5/95 [00:21<06:22,  4.25s/it, Loss=1.02, MAE=4.56e+3, MAPE=7.2
Training:   5%| | 5/95 [00:25<06:22,  4.25s/it, Loss=0.991, MAE=4427.03, MAPE=3.
Training:   6%| | 6/95 [00:25<06:19,  4.26s/it, Loss=0.991, MAE=4427.03, MAPE=3.
Training:   6%| | 6/95 [00: